In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
Xtrain = pickle.load(open('Xtrain_ANN.data', 'rb'))
Ytrain = pickle.load(open('Ytrain_ANN.data', 'rb'))

In [3]:
from sklearn.decomposition import PCA

In [4]:
pca = PCA(n_components=100)

In [5]:
pca.fit(Xtrain)

PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [6]:
pca.explained_variance_

array([1.53691052, 0.51650046, 0.49245962, 0.44320213, 0.3996141 ,
       0.3728763 , 0.34904255, 0.30409751, 0.28935477, 0.22907134])

In [7]:
pca.singular_values_

array([952.68336905, 552.28042499, 539.27415161, 511.59373042,
       485.78566607, 469.25259601, 454.00797359, 423.77050321,
       413.3706272 , 367.79825819])

In [8]:
Xtrain = pd.DataFrame(data=pca.transform(Xtrain))
Xtrain.head()

,0,1,2,3,4,5,6,7,8,9
0,0.147497,-0.128136,-0.555639,0.292194,-0.168896,1.501090,0.938601,0.363683,-0.396043,-0.006755
1,0.215837,-0.316249,-0.480825,1.065285,-0.388673,0.314881,0.001644,-0.663229,-1.176961,0.081666
2,-1.357071,0.254334,0.212074,-0.170978,0.917640,-0.101092,0.896093,0.363979,0.523785,-0.618507
3,-0.699646,0.090907,-0.430041,0.786888,-0.922471,-0.422977,-0.176328,0.432024,0.212012,-0.075511
4,2.361831,0.017103,1.310205,1.881892,-0.120687,-0.377364,0.510773,-0.666712,-0.220328,0.107476


In [10]:
Xtrain.shape

(590540, 10)

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1, criterion='entropy')

In [14]:
clf.fit(Xtrain, Ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [15]:
from sklearn.metrics import log_loss, classification_report, confusion_matrix

In [16]:
Ytrain_ = clf.predict(Xtrain)

In [17]:
print(log_loss(Ytrain, Ytrain_))
print(confusion_matrix(Ytrain, Ytrain_))
print(classification_report(Ytrain, Ytrain_))

9.99200722162641e-16
[[569877      0]
 [     0  20663]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    569877
           1       1.00      1.00      1.00     20663

    accuracy                           1.00    590540
   macro avg       1.00      1.00      1.00    590540
weighted avg       1.00      1.00      1.00    590540



In [18]:
Xtest = pickle.load(open('Xtest_ANN.data', 'rb'))

In [19]:
Xtest = pca.transform(Xtest)

In [20]:
Ytest_ = clf.predict_proba(Xtest)

In [21]:
submission = pd.read_csv('./raw/sample_submission.csv', index_col='TransactionID')

In [22]:
submission.count()

isFraud    506691
dtype: int64

In [23]:
submission['isFraud'] = Ytest_
submission.head()

,isFraud
TransactionID,
3663549,1.000
3663550,0.980
3663551,0.982
3663552,0.992
3663553,0.996


In [24]:
submission['isFraud'] = 1 - submission['isFraud']

In [25]:
submission[submission['isFraud'] > 0.5].count()

isFraud    1228
dtype: int64

In [26]:
submission.to_csv('predicted_PCA_RFCv1.csv')

In [27]:
pickle.dump(clf, open('clf_PCA_RFC.model', 'wb'))